# RAG 증강, 생성(Retrieval, Augmented,  Generation)

## 환경 변수 로딩

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

## 필요 라이브러리 로딩

In [2]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

## 검색

In [12]:
def sm_search_top_k(question):
    load_dotenv(override=True)
    
    
    #벡터 DB, index 객체 생성
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "wiki"
    index = pc.Index(index_name)
    
    # embedding 객체 생성
    embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = OPENAI_API_KEY
    )
    
    # pineconevectorstore 객체 생성
    vector_store = PineconeVectorStore(index=index,        
                                   embedding=embedding, 
                                   text_key="full_text"
                                  )
    
    
    results = vector_store.similarity_search(
    query=question, # question 문자열을 embedding vector로 자동 변환 됨.
    k=3, 
    namespace="wiki_ns1")

    context = ""
    for result in results:
        context = context + result.page_content + f"\n{'-'*30}\n"
    # print(result.metadata)
    # print(result.page_content)
    # print("-"*30)
    
    return context

### 질문 top_k 검색

In [4]:
# question = ["김대중 대통령의 행적에 대해 알여줘"]
# top_k_context = sm_search_top_k(question)
# print(top_k_context)

## 증강

### 프롬프트 템플릿 만들기

In [41]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    #시스템 프롬프트
    ("system", "당신은 역사전문가 특히 인물에 대해 상당한 조예가 깊습니다. 질문에 대해 최적의 답을 해주세요."),
    # user prompt
    ("human", """
    {question}에 대해 설명해 주세요.
                 아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요. 없으면 없다고 말해주세요.
                 context:{context}"""
    )])

chat_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 역사전문가 특히 인물에 대해 상당한 조예가 깊습니다. 질문에 대해 최적의 답을 해주세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    {question}에 대해 설명해 주세요.\n                 아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요. 없으면 없다고 말해주세요.\n                 context:{context}'), additional_kwargs={})])

### 질문 만들기

In [18]:
question = "링컨대통령에 대해 설명해줘."
# question = "백남준에 대해 설명해줘."

### 질문에 대한 topk 만들기

In [19]:
top_k_context = sm_search_top_k(question)

In [20]:
print(top_k_context)

평가 
경제문제를 해결하지 못하고 주 이란 미국 대사관 인질 사건에 발목이 잡혀 실패한 대통령으로 평가를 받지만 이란 사태는 미국 내 이란 재산을 풀어주겠다는 조건을 내세워서 사실상 카터가 해결한 것이었고, 사랑의 집짓기 운동 등으로 퇴임 후에 훨씬 더 존경받는 미국 대통령 중에 특이한 인물로 남았다.

그는 2002년 말 인권과 중재 역할에 대한 공로를 인정받아 노벨 평화상을 받게 되었다.

이외에도, 그는 대통령 재임 시절은 물론 퇴임 후에도 지속적으로 여러 장기 집권중인 독재자들을 만나왔는데, 그와 만난 독재자들 중 절대 다수가 얼마 되지 않아 최후를 맞이하게 되며 '독재자의 사신'이라는 별명이 붙기도 했다.

같이 보기 
 주한 미군의 철수
 한반도 평화협정

역대 선거 결과

각주

참고 문헌 
 《진정한 리더는 떠난 후에 아름답다》 저자 : 지미 카터
 《지미 카터》 저자 : 지미 카터(지식의날개, 2018)

외부 링크
------------------------------
반면 제1야당 한나라당는 선거에서 패배하며 정권 교체를 실패하며 계속 야당시절을 보내게 된다. 이회창 은퇴와 참여정부의 출범 이후 원내 의석수을 악용해 압박하여 참여정부에게 향한 견제를 하기 시작했지만 그러나 2003년 11월 한나라당이 대기업들부터 선거 자금을 받았다는 의혹로 인해 16대 대선 차떼기 사건으로 역풍을 맞게되고 한나라당는 창당 이래 최악의 위기를 맞게된다. 하지만 2004년 3월 17대 총선를 한달 앞두고 제2야당 새천년민주당, 소수야당 자민련와의 공조하여 노무현 대통령의 탄핵을 일으켜 민심의 역풍을 맞게되었지만 17대 총선에서 16년 만에 원내 1당 지위를 상실했지만 개헌 저지선 100석 이상 사수을 하며 선전하는데 만족했다. 이회창 은퇴와 한나라당의 실질적 뿌리 민정계 인사들의 퇴장 이후 박정희 대통령의 차녀 박근혜 대표와 이명박 서울시장의 등장으로 친박계과 친이계이란 거대한 파벌이 등장했다. 이후 두 사람는 17대 대선과 18대 대선에서 나란히 승리하여
-

### 최종 프롬프트 만들기
- 템플릿 프롬프트 + 사용자 질문 + top_k context

In [21]:
prompt = chat_template.format_messages(
    question = question,
    context = top_k_context
)

prompt

[SystemMessage(content='당신은 역사전문가 특히 인물에 대해 상당한 조예가 깊습니다. 질문에 대해 최적의 답을 해주세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="\n    링컨대통령에 대해 설명해줘.에 대해 설명해 주세요.\n                 아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요. 없으면 없다고 말해주세요.\n                 context:평가 \n경제문제를 해결하지 못하고 주 이란 미국 대사관 인질 사건에 발목이 잡혀 실패한 대통령으로 평가를 받지만 이란 사태는 미국 내 이란 재산을 풀어주겠다는 조건을 내세워서 사실상 카터가 해결한 것이었고, 사랑의 집짓기 운동 등으로 퇴임 후에 훨씬 더 존경받는 미국 대통령 중에 특이한 인물로 남았다.\n\n그는 2002년 말 인권과 중재 역할에 대한 공로를 인정받아 노벨 평화상을 받게 되었다.\n\n이외에도, 그는 대통령 재임 시절은 물론 퇴임 후에도 지속적으로 여러 장기 집권중인 독재자들을 만나왔는데, 그와 만난 독재자들 중 절대 다수가 얼마 되지 않아 최후를 맞이하게 되며 '독재자의 사신'이라는 별명이 붙기도 했다.\n\n같이 보기 \n 주한 미군의 철수\n 한반도 평화협정\n\n역대 선거 결과\n\n각주\n\n참고 문헌 \n 《진정한 리더는 떠난 후에 아름답다》 저자 : 지미 카터\n 《지미 카터》 저자 : 지미 카터(지식의날개, 2018)\n\n외부 링크\n------------------------------\n반면 제1야당 한나라당는 선거에서 패배하며 정권 교체를 실패하며 계속 야당시절을 보내게 된다. 이회창 은퇴와 참여정부의 출범 이후 원내 의석수을 악용해 압박하여 참여정부에게 향한 견제를 하기 시작했지만 그러나 2003년 11월 한나라당이 대기업들부터 선거 자금을 받았다는 의혹로 인해 16대 대선 차떼기 사건으로 역풍을 맞게되고 한나라당는 창당 

## 생성
- llm이 프롬프트에 따라 결과를 생성함.

In [22]:
# ChatOpenAI 객체를 초기화
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

In [23]:
response = llm.invoke(prompt)
print(response.content)

죄송하지만, 제공된 context에는 링컨 대통령에 대한 정보가 포함되어 있지 않습니다. 링컨에 관한 설명이나 정보는 제공할 수 없습니다.


## LCEL 로 만들어 사용하기

In [34]:
from langchain_core.output_parsers import JsonOutputParser

In [35]:
# parser = JsonOutputParser

In [48]:
question = "백남준에 대해 설명해줘."

In [49]:
top_k_context = sm_search_top_k(question)
print(top_k_context)

백남준(白南準, 1932년 7월 20일 ~ 2006년 1월 29일, 서울 출생, )은 한국 태생의 세계적인 비디오 아트 예술가, 작곡가, 전위 예술가이다. 본관은 수원(水原).

생전에 뉴욕, 쾰른, 도쿄, 마이애미와 서울에 주로 거주한 그는 여러 가지 매체로 예술 활동을 하였다. 특히 비디오 아트라는 새로운 예술의 범주를 발전시켰다는 평가를 받는 예술가로서 '비디오 아트의 창시자'로 알려져 있다.

생애
------------------------------
《청년, 백남준: 초기 예술의 융합 미학》
연구자 임산(현 동덕여대 큐레이터학과 교수) 씨의 연구서로서 단행본 출간.
 《백남준을 말하다》
국내의 백남준 지인들이 각자 얽힌 인연을 바탕으로 나름의 해석을 담은 책 출간.
 박정진. 《(eBOOK)굿으로 보는 백남준 비디오아트 읽기》. 한국학술정보. 2014년.
------------------------------
관련 영상 
 백남준의 위성예술 1부 (1984년 방송)
 백남준의 위성예술 2부 (1984년 방송)
 백남준의 비디오아트세계 (1984년 방송)
 백남준, 예술로 미래를 말하다 (2006년 방송) - 백남준 추모특집

각주

외부 링크 

  백남준아트센터
 ARTCYCLOPEDIA에서 백남준 항목
 백남준의 세계 (출처:Guggenheim.org)
 백남준 간략한 일대기와 주요 작품 설명 (출처:Electronic Arts Intermix)
 백남준 일대기 (출처:@ MedienKunstNetz)}
 백남준 부고 "Father of Video Art Paik Nam-june Dies", (출처:조선일보, 2006년 1월 30일자 신문)
 백남준 부고 "Video artist Nam June Paik dead at 74", (출처:뉴욕 타임즈, 2006년 1월 30일 보도)
 백남준 공식 블로그
------------------------------



In [50]:
input_data = {"question" : question ,"context": top_k_context}

In [51]:
# 실행 파이프라인 구성

chain = chat_template | llm 



In [52]:
response = chain.invoke(input_data)

In [53]:
result = response.content
print(result)

백남준(1932년 7월 20일 ~ 2006년 1월 29일)은 한국 태생의 세계적인 비디오 아트 예술가, 작곡가, 전위 예술가로, 본관은 수원입니다. 그는 뉴욕, 쾰른, 도쿄, 마이애미, 서울 등에서 주로 거주하며 다양한 매체로 예술 활동을 펼쳤습니다. 백남준은 '비디오 아트의 창시자'로 알려져 있으며, 이 새로운 예술 범주를 발전시킨 것으로 평가받고 있습니다. 그의 작품은 현대 예술에 큰 영향을 미쳤으며, 생전에는 비디오 아트를 통해 미래에 대한 메시지를 전달하려고 노력했습니다.
